In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests


In [363]:
path = r"C:\Users\Colmj\Desktop\UL_PHD_COURSEWORK\UL_Cancer_Network_Project\Information_Validation_System\geckodriver.exe"
driver = webdriver.Firefox(executable_path= path)
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0'}

In [364]:
driver.get("https://scholar.google.com/scholar?start=900&q=%22colon+cancer%22&hl=en&as_sdt=0,5&as_ylo=2022&as_yhi=2022")

In [257]:
import random

In [329]:
#all_bibs = []

1.882029521928374

In [366]:
while True:
    #WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'gs_or_cit gs_or_btn gs_nph')))
    cite_buttons = driver.find_elements_by_xpath('//a[@class="gs_or_cit gs_or_btn gs_nph"]')
    for cite_button in cite_buttons:
        cite_button.send_keys(Keys.ENTER)
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'gs_citi')))
        bibtex_link = driver.find_elements_by_xpath('//a[@class="gs_citi"]')[0]
        res = requests.get(bibtex_link.get_attribute("href"), headers=headers)
        if res.status_code == 200: 
            bibtex = res.content.decode()
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        time.sleep(abs(random.normalvariate(20,5)))
        all_bibs.append(bibtex)
    try:
        next_link = driver.find_elements_by_xpath('//span[@class="gs_ico gs_ico_nav_next"]')[0].find_element_by_xpath('..').get_attribute("href")
        time.sleep(abs(random.normalvariate(10,3)))
        driver.get(next_link)
        #Hitting the next button - if none exists must be at the end therefore breakloop
        #WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.TAG_NAME, 'gs_or_cit gs_or_btn gs_nph')))
    except IndexError as e:
        print(e)
        break
    
#driver.close()

list index out of range


In [160]:
driver.execute_script('''window.open("{}","_blank");'''.format(bibtex_link.get_attribute("href")))

In [250]:
driver.get_screenshot_as_file("img.png")

True

In [5]:
from collections import Counter

In [375]:
unique_bibs = set(all_bibs)

In [188]:
driver.close()

In [163]:
cite_buttons[1]

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="88f5ece4-bf5d-4dd4-ab1c-7588c835da4a", element="357c9f6c-bc4c-4c4e-9877-1e1ab1575e6b")>

In [164]:
cite_buttons[1].send_keys(Keys.ENTER)
WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'gs_citi')))
bibtex_link = driver.find_elements_by_xpath('//a[@class="gs_citi"]')[0]

In [186]:
import dill as pickle

In [378]:
with open("bibtext_list.pkl", "wb") as f:
    pickle.dump(all_bibs, f)

In [187]:
with open("bibtext_list.pkl", "rb") as f:
    all_bibs = pickle.load(f)

In [188]:
unqiue_bibs = list(set(all_bibs))

In [39]:
breakpoints = [(0,269), (269, len(unqiue_bibs))]

In [40]:
for bp in breakpoints:
    with open('bibtex_files/references_{}_{}.bib'.format(bp[0],bp[1]), 'w', encoding="utf-8") as bibfile:
        bibtext_string = " ".join(all_bibs[bp[0]:bp[1]])
        bibfile.write(bibtext_string)

In [189]:
unqiue_bibs 

['@article{harbscirculating,\n  title={Circulating Sex Hormone Levels and Colon Cancer Risk in Men: A Nested Case-Control Study and Meta-analysisCirculating Sex Hormone Levels and Colon Cancer Risk in Men},\n  author={Harbs, Justin and Rinaldi, Sabina and Gicquiau, Audrey and Keski-Rahkonen, Pekka and Mori, Nagisa and Liu, Xijia and Kaaks, Rudolf and Katzke, Verena and Schulze, Matthias B and Agnoli, Claudia and others},\n  journal={Cancer Epidemiology, Biomarkers \\& Prevention},\n  publisher={AACR}\n}\n',
 '@article{roufarshbaf2022comparison,\n  title={Comparison of four methods of colon cancer cell lysates preparation for ex vivo maturation of dendritic cells},\n  author={Roufarshbaf, Mohammad and Esmaeil, Nafiseh and Akbari, Vajihe},\n  journal={Research in Pharmaceutical Sciences},\n  volume={17},\n  number={1},\n  pages={43},\n  year={2022},\n  publisher={Wolters Kluwer--Medknow Publications}\n}\n',
 '@article{bennedsen2022exploration,\n  title={An exploration of immunohistochemi

In [6]:
import textract
from more_itertools import windowed
from IPython.display import clear_output
from os import listdir, name
from os.path import isfile, join
import re

In [177]:
mypath = "reference_pdfs"       
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [182]:
onlyfiles[0]

['references', '0', 'apa']

In [7]:
from fuzzywuzzy import fuzz
import numpy as np

2

In [218]:
fuzz.partial_ratio("hello my name is john", "my name is david")

71

In [232]:
all_references = []
all_labels = []
for file in onlyfiles:
    reference_style = file[:-4].split("_")[-1]
    text = textract.process("reference_pdfs/references_0_apa.pdf")
    text_as_string = text.decode("utf-8")
    #Splits the text up into pages
    refs = text_as_string.split("\x0c")

    # Interactive element
    join_or_not = []
    for window in windowed(refs,2):
        clear_output(wait=True)
        print(window[0][-20:])
        print(window[1][:20])
        join_or_not.append(input())

    refs_processed = []
    temp_list = []
    for join_command, window in zip(join_or_not, windowed(refs,2)):
        #print(temp_list, window, join_command)
        if join_command == 'n':
            if len(temp_list) > 0:
                refs_processed.append(" ".join(temp_list))
                temp_list = []
            else:
                refs_processed.append(window[0].strip())
        else:
            for pane in window:
                if pane.strip() not in temp_list:
                    temp_list.append(pane.strip())
    if len(temp_list) > 0 :
        print("I was called")
        refs_processed.append(" ".join(temp_list))
        
    individual_references = []
    for ref_chunk in refs_processed:
        for reference in ref_chunk.split("\n"):
            individual_references.append(reference)

    individual_labels = []

    #Using Levensthein distance to match formatted reference to bibtex for labelling as string matching did not work reliably
    for ref in individual_references:
        ref_to_bib_distances = []
        for bib in unqiue_bibs:
            bib_title = re.findall(r'title={([^}]*)}',bib)[0]
            ref_to_bib_distances.append(fuzz.partial_ratio(ref,bib_title))
        individual_labels.append((reference_style,unqiue_bibs[np.argmax(ref_to_bib_distances)].split("{")[0][1:]))
       
    
    all_references.extend(individual_references)
    all_labels.extend(individual_labels)

    



rs, 14 (3), 784.


Cao, H., Sethumadhav


KeyboardInterrupt: Interrupted by user

In [9]:
import dill as pickle

In [30]:
with open("all_references.pkl", "rb") as f:
    all_references = pickle.load(f)

with open("all_labels.pkl", "rb") as f:
    all_labels = pickle.load(f)

In [248]:
to_remove = [] 

In [18]:
to_merge = []

In [253]:
to_remove.sort(reverse=True)

In [29]:
for tm in to_merge:
    all_references.pop(tm)
    all_labels.pop(tm)

IndexError: pop index out of range

In [28]:
to_merge.sort(reverse=True)

In [27]:
for i, ref in enumerate(all_references):
        if len(ref) < 20:
            to_merge.append(i)
            print(i, ref)
            print()

0 References

71 References

189 References

307 References

425 References

543 References

660 References

778 References

896 References

1014 Works Cited

1131 References

1248 References

1365 References

1482 References

1595 References

1712 References



In [59]:
clean_labels = all_labels[:1856]

In [49]:
clean_references = all_references[:1856]

In [53]:
to_merge = []

In [56]:
to_merge.sort(reverse=True)

In [60]:
for tm in to_merge:
    clean_labels.pop(tm)
    clean_references.pop(tm)

In [63]:
for i,cr in enumerate(clean_references):
    if len(cr) < 25:
        to_merge.append(i)
        print(cr)

In [338]:
reference_titles.sort(reverse=True)

In [79]:
import os 
from bs4 import BeautifulSoup

In [80]:
documents = []
for file in os.listdir("reuters/"):
    if file.endswith(".sgm"):
        # for each sgm file, read it
        filename = os.path.join("reuters", file)
        f = open(filename, 'r', encoding='utf-8', errors='ignore')
        dataFile = f.read()
        
        # pass it to BeautifulSoup
        soup = BeautifulSoup(dataFile, 'html.parser')
        contents = soup.findAll('body')
        
        # for each body tag, extract it's text
        for content in contents:
            documents.append(content.text)
            print('We have {} documents'.format(len(documents)))
print(documents[0])

We have 1 documents
We have 2 documents
We have 3 documents
We have 4 documents
We have 5 documents
We have 6 documents
We have 7 documents
We have 8 documents
We have 9 documents
We have 10 documents
We have 11 documents
We have 12 documents
We have 13 documents
We have 14 documents
We have 15 documents
We have 16 documents
We have 17 documents
We have 18 documents
We have 19 documents
We have 20 documents
We have 21 documents
We have 22 documents
We have 23 documents
We have 24 documents
We have 25 documents
We have 26 documents
We have 27 documents
We have 28 documents
We have 29 documents
We have 30 documents
We have 31 documents
We have 32 documents
We have 33 documents
We have 34 documents
We have 35 documents
We have 36 documents
We have 37 documents
We have 38 documents
We have 39 documents
We have 40 documents
We have 41 documents
We have 42 documents
We have 43 documents
We have 44 documents
We have 45 documents
We have 46 documents
We have 47 documents
We have 48 documents
W

In [62]:
lengths = []

for ref in clean_references:
    sents = nltk.sent_tokenize(ref)
    lengths.append(len(sents))

86

In [64]:
import random
import numpy as np

In [1]:
clean_references

NameError: name 'clean_references' is not defined

In [353]:
documents[0].split(".\n")

['Showers continued throughout the week in\nthe Bahia cocoa zone, alleviating the drought since early\nJanuary and improving prospects for the coming temporao,\nalthough normal humidity levels have not been restored,\nComissaria Smith said in its weekly review',
 '    The dry period means the temporao will be late this year',
 '    Arrivals for the week ended February 22 were 155,221 bags\nof 60 kilos making a cumulative total for the season of 5.93\nmln against 5.81 at the same stage last year. Again it seems\nthat cocoa delivered earlier on consignment was included in the\narrivals figures',
 '    Comissaria Smith said there is still some doubt as to how\nmuch old crop cocoa is still available as harvesting has\npractically come to an end. With total Bahia crop estimates\naround 6.4 mln bags and sales standing at almost 6.2 mln there\nare a few hundred thousand bags still in the hands of farmers,\nmiddlemen, exporters and processors',
 '    There are doubts as to how much of this coc

In [93]:
len(clean_labels)

1697

In [94]:
samples = []
sample_labels = []
for document in documents:
    sents = document.split(".\n")
    for sent in sents:
        samples.append(sent)
        sample_labels.append("sentence")



104851

In [97]:
with open("sentence_samples.pkl", "wb") as f:
    pickle.dump(samples,f)

with open("sentence_sample_labels.pkl", "wb") as f:
    pickle.dump(sample_labels,f)

In [99]:
with open("reference_samples.pkl", "wb") as f:
    pickle.dump(clean_references,f)

with open("reference_sample_labels.pkl", "wb") as f:
    pickle.dump(clean_labels,f)